In [84]:
import pandas as pd
import numpy as np
import math

In [85]:
articles = pd.read_csv('data/shared_articles.csv')
interactions = pd.read_csv('data/users_interactions.csv')

In [86]:
articles.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


Информация о данных:
- дата публикации (временная метка),
- исходный URL-адрес,
- заголовок,
- содержание в виде обычного текста,
- язык статьи (португальский — pt или английский — en),
- информация о пользователе, который поделился статьёй (автор).

In [87]:
interactions.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


Информация о событиях:
- VIEW — просмотр,
- LIKE — лайк,
- COMMENT CREATED — комментарий,
- FOLLOW — подписка,
- BOOKMARK — добавление в закладки.

Отфильтруем данные так, чтобы остались только объекты с типом события CONTENT SHARED.

In [88]:
articles = articles[articles['eventType'] == 'CONTENT SHARED']
articles.shape

(3047, 13)

In [89]:
interactions.personId = interactions.personId.astype(str)
interactions.contentId = interactions.contentId.astype(str)
articles.contentId = articles.contentId.astype(str)

Создадим универсальный индекс популярности. Составим его из реакций пользователей, придав им разные веса.

In [90]:
event_type_index = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

Создадим признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами).

In [91]:
interactions['eventIndex'] = interactions.eventType.apply(lambda x: event_type_index[x])
interactions['eventIndex'].mean()

1.2362885828078327

Для минимизации проблемы холодного старта оставим только тех пользователей, которые взаимодействовали хотя бы с пятью статьями.

In [92]:
users_interactions = (
    interactions
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId')
    .size()
)
users_interactions.head()

personId
-1007001694607905623      6
-1032019229384696495    648
-108842214936804958     270
-1093393486211919385      2
-1110220372195277179      3
dtype: int64

In [93]:
users_5_interactions = users_interactions[users_interactions >= 5].reset_index()[['personId']]
print(len(users_5_interactions))

1140


Оставим только те взаимодействия, которые касаются только отфильтрованных пользователей (то есть тех, которые взаимодействовали как минимум с пятью статьями).

In [94]:
intersect_df = interactions.loc[np.in1d(interactions.personId, users_5_interactions)]
print(intersect_df.shape)

(69868, 9)


Прологарифмируем сумму весов для взаимодействия пользователя с каждой конкретной статьёй. Также сохраните для каждой пары «пользователь — статья» значение времени последнего взаимодействия.

In [95]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

In [96]:
df = (
    intersect_df
    .groupby(['personId', 'contentId'])
    .eventIndex.sum()
    .apply(smooth_user_preference)
    .reset_index()
    .set_index(['personId', 'contentId'])
)

In [97]:
df['last_timestamp'] = (
    intersect_df
    .groupby(['personId', 'contentId'])
    .timestamp.last()
)
df = df.reset_index()

In [98]:
df['last_timestamp'].mean()

1470587338.35212

Разделим данные на обучающую и тестовую выборки, выбрав в качестве временной отсечки значение 1475519545.

In [99]:
split_ts = 1475519545
train_df = df.loc[df.last_timestamp < split_ts].copy()
test_df = df.loc[df.last_timestamp >= split_ts].copy()
 
print(len(train_df))

29329


# Часть I. Рекомендательнаяя система на основе популярности.

Преобразуем данные так, чтобы получить таблицу в формате, где строка соответствует пользователю, а столбцы будут истинными предпочтениями и рекомендациями в формате списков. На место пустых ячеек поместим пустые списки.

In [100]:
final_df = (
    train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    test_df.groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]

Посчитаем популярность каждой статьи как сумму всех «оценок» взаимодействий с ней, используя только обучающую выборку.

In [101]:
popular = (
    train_df
    .groupby('contentId')
    .eventIndex.sum().reset_index()
    .sort_values('eventIndex', ascending=False)
    .contentId.values
)
popular[0]

'-6783772548752091658'

Сформируем рекомендации для каждого пользователя. Будем рекомендовать десять самых популярных статей. Будем предлагать пользователю только то, что он ещё не читал.

Оценим качество с помощью precision@10 для каждого пользователя (доля угаданных рекомендаций). После этого усредним результат по всем пользователям.

In [102]:
def precision(column):
    return (
        final_df.apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)
        ).mean()

In [103]:
top_k = 10
 
final_df['popular'] = (
    final_df.true_train
    .apply(
        lambda x:
        popular[~np.in1d(popular, x)][:top_k]
    )
)

In [104]:
final_df

,true_train,true_test,popular
personId,,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72...","[-6783772548752091658, -133139342397538859, -8..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -...","[-6783772548752091658, -133139342397538859, -8..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -...","[-6783772548752091658, -8208801367848627943, 8..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -...","[-133139342397538859, -8208801367848627943, 82..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224],"[-6783772548752091658, -8208801367848627943, 8..."
...,...,...,...
953707509720613429,"[-1068603220639552685, -2358756719610361882, -...","[-2402288292108892893, -5813211845057621660, -...","[-133139342397538859, -8208801367848627943, 82..."
983095443598229476,"[-133139342397538859, -8742648016180281673]","[-14569272361926584, -1572252285162838958, -18...","[-6783772548752091658, -8208801367848627943, 8..."
989049974880576288,"[-133139342397538859, -2038869595290705317, -2...","[-6289909056857931861, -7047448754687279385, -...","[-6783772548752091658, -8208801367848627943, 8..."


In [105]:
precision('popular')

0.006454207722621089

Качество получилось не очень высоким, так как рекомендации были неперсонализированными.

# Часть II. Рекомендательнаяя система на основе более сложных алгоритмов.

Построим матрицу, в которой по столбцам будут находиться id статей, по строкам — id пользователей, а на пересечениях строк и столбцов — оценка взаимодействия пользователя со статьёй. Если взаимодействия не было, в соответствующей ячейке поставим ноль.

Найдите оценку взаимодействия пользователя с ID -1032019229384696495 со статьёй с ID 943818026930898372.

In [106]:
ratings = pd.pivot_table(
    train_df,
    values='eventIndex',
    index='personId',
    columns='contentId').fillna(0)
ratings.loc['-1032019229384696495','943818026930898372']

2.321928094887362

Применим memory-based-подход коллаборативной фильтрации. Для увеличения скорости работы преобразуем таблицу в массив numpy.

In [107]:
ratings_np = ratings.values
ratings_np.mean()

0.016673743043640697

#### Реализуем коллаборативную фильтрацию.

Построим матрицу схожести. Для этого вычислим все попарные коэффициенты корреляции для матрицы, полученной выше. Для каждой пары будем учитывать только ненулевые значения (так как нулевые обозначают отсутствие взаимодействия и не интересуют нас).

In [108]:
similarity_users = np.zeros((len(ratings_np), len(ratings_np)))
for i in range(len(ratings_np)-1):
    for j in range(i+1, len(ratings_np)):
        mask_uv = (ratings_np[i] != 0) & (ratings_np[j] != 0)
        ratings_v = ratings_np[i, mask_uv]
        ratings_u = ratings_np[j, mask_uv]
        similarity_users[i,j] = np.corrcoef(ratings_v, ratings_u)[0, 1]
        similarity_users[j,i] = similarity_users[i,j]

c:\Users\krygl\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
c:\Users\krygl\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
c:\Users\krygl\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2821: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\krygl\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\krygl\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2680: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
c:\Users\krygl\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib

Выведите результат, полученный в ячейке с третьим индексом по строкам и сороковым — по столбцам.

In [109]:
similarity_users[3,40]

-0.3333333333333333

Для построения рекомендаций реализуем алгоритм.

Для каждого пользователя:

- Найдем пользователей с похожестью больше 0.
- Для каждой статьи вычислим долю пользователей (среди выделенных на первом шаге), которые взаимодействовали со статьёй.
- Порекомендуем статьи с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел).

In [110]:
interactions_df = (
    train_df.groupby("personId")["contentId"]
    .agg(list)
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions_df["true_test"] = test_df.groupby("personId")[
    "contentId"
].agg(list)

interactions_df["true_test"] = [[] if x is np.NaN else x for x in interactions_df["true_test"]]
interactions_df.head()

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224]


In [111]:
prediction_user_based = []
for i in range(len(similarity_users)):
    users_sim = similarity_users[i] > 0
    if len(users_sim) == 0:
        prediction_user_based.append([])
    else:
        tmp_recommend = np.argsort(ratings_np[users_sim].sum(axis=0))[::-1]
        tmp_recommend = ratings.columns[tmp_recommend]
        recommend = np.array(tmp_recommend)[~np.in1d(tmp_recommend, final_df.iloc[i])][:10]
        prediction_user_based.append(list(recommend))
interactions_df['prediction_user_based'] = prediction_user_based

Найдите первую рекомендацию для строки 34 (если считать с нуля).

In [112]:
prediction_user_based[34][0]

'98528655405030624'

Вычислим качество по метрике, которую мы определили выше при решении этой задачи.

In [113]:
def calc_precision(column):
    return (
        interactions_df.apply(
            lambda row:
                len(set(row['true_test']).intersection(
                set(row[column]))) /
                min(len(row['true_test']) + 0.001, 10.0), 
                axis=1)
        ).mean()

In [114]:
calc_precision('prediction_user_based')

0.003541692918885616

Реализуем рекомендательную систему с использованием SVD. Разложим матрицу взаимодействий пользователей со статьями с помощью функции svd из модуля scipy.

In [115]:
from scipy.linalg import svd

U, sigma, V = svd(ratings)

Найдите максимальное значение в получившейся матрице U.

In [116]:
U.max()

0.7071067811865486

Значения матрицы с сингулярными числами отсортированы по убыванию. Допустим, мы хотим оставить только первые 100 компонент и получить скрытые представления размерности 100. Для этого необходимо оставить 100 столбцов в матрице U, только первые 100 значений из sigma (и сделать из них диагональную матрицу) и 100 столбцов в матрице V. Затем необходимо перемножить преобразованные матрицы.

Найдите сумму всех элементов в новой сингулярной матрице.

In [117]:
k = 100
s = sigma[:k]
U = U[:, 0:k]
V = V[0:k, :]
s = np.diag(s)
round(s.sum(), 2)

2096.53

In [118]:
print(U.shape)
print(s.shape)
print(V.shape)

(1112, 100)
(100, 100)
(100, 2366)


Cделаем предсказание по полученной матрице. Найдем для каждого пользователя статьи с наибольшими оценками в восстановленной матрице.

In [119]:
new_ratings = U.dot(s).dot(V)
new_ratings = pd.DataFrame(new_ratings, index=ratings.index, columns=ratings.columns)
top_k = 10
predictions = []

In [120]:
for personId in interactions_df.index:
    prediction = (
        new_ratings
        .loc[personId]
        .sort_values(ascending=False)
        .index.values
    )
    
    predictions.append(
        list(prediction[~np.in1d(
            prediction,
            interactions_df.loc[personId, 'true_train'])])[:top_k])

interactions_df['prediction_svd'] = predictions

calc_precision('prediction_svd')

0.012212989310270756

Реализуем на гибридную модель и посмотрим, какое качество получится. Для этого воспользуемся библиотекой LightFM.

In [122]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k
from scipy.sparse import csr_matrix

ratings_matrix = csr_matrix((ratings))

model = LightFM(loss='warp', #определяем функцию потерь
                random_state=13, #фиксируем случайное разбиение
                learning_rate=0.05, #темп обучения
                no_components=100) #размерность вектора для представления данных в модели

train,test = random_train_test_split(ratings_matrix, test_percentage=0.3, random_state=13)
model = model.fit(train)
prec_score = precision_at_k(model, test).mean()

print(prec_score)

0.03764706
